## Gerekli Kütüphanelerin Yüklenmesi

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import seaborn as sns

# Matplotlib grafiklerini notebook içinde göstermek için
%matplotlib inline

# Grafik stilini belirle
sns.set(style="whitegrid")

## Veri Setini Yükleme ve İlk İnceleme

In [ ]:
# Veri setini yükle
data = pd.read_csv('train_odev.csv')

# Veri setinin ilk 5 satırına göz at
data.head()

## Veri Setinin Genel Bilgileri ve Eksik Veri Kontrolü

In [ ]:
# Min-Max ölçekleme ile 'memory' sütununu ölçekle
scaler = MinMaxScaler()
data['memory_scaled'] = scaler.fit_transform(data[['memory']])

# 'speed' sütununun ortalamasına göre yeni bir özellik oluştur
data['high_speed'] = (data['speed'] > data['speed'].mean()).astype(int)

# Gerekli olmayan sütunları çıkar
X = data.drop(columns=['price_range', 'speed', 'memory'])
y = data['price_range']

# Ön işlenmiş verinin ilk 5 satırını görüntüle
X.head()

## Veri Görselleştirme (Histogramlar ve Dağılım Grafikleri)

In [ ]:
# Histogram ile bazı özelliklerin dağılımını incele
plt.figure(figsize=(12, 8))
data[['memory_scaled', 'speed', 'high_speed']].hist(bins=20, figsize=(12, 8))
plt.tight_layout()
plt.show()

## Veriyi Eğitim ve Test Setlerine Ayırma

In [ ]:
# Veriyi eğitim ve test setlerine ayır
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(f"Eğitim seti boyutu: {X_train.shape}")
print(f"Test seti boyutu: {X_test.shape}")

## Random Forest ve GridSearchCV ile Model Eğitimi

In [ ]:
# Hiperparametre aralığı
parametre_alanı = {
    'n_estimators': [50, 100, 200],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Random Forest modeli
rf_model = RandomForestClassifier(random_state=42)

# GridSearchCV ile en iyi parametreleri bul
grid_arama = GridSearchCV(estimator=rf_model, param_grid=parametre_alanı, cv=3, n_jobs=-1, scoring='accuracy')
grid_arama.fit(X_train, y_train)

# En iyi modeli al
en_iyi_rf_modeli = grid_arama.best_estimator_

print("En iyi hiperparametreler:", grid_arama.best_params_)

## Model Performansını Değerlendirme

In [ ]:
# Modelin test seti üzerinde tahmini
y_tahmin = en_iyi_rf_modeli.predict(X_test)

# Doğruluk skoru ve sınıflandırma raporu
doğruluk = accuracy_score(y_test, y_tahmin)
sınıflandırma_raporu = classification_report(y_test, y_tahmin)
karmaşıklık_matrisi = confusion_matrix(y_test, y_tahmin)

print("Doğruluk:", doğruluk)
print("\nSınıflandırma Raporu:\n", sınıflandırma_raporu)

## Karmaşıklık Matrisini Görselleştirme

In [ ]:
# Karmaşıklık matrisini görselleştir
plt.figure(figsize=(8, 6))
sns.heatmap(karmaşıklık_matrisi, annot=True, fmt='d', cmap='Blues', xticklabels=y.unique(), yticklabels=y.unique())
plt.title('Karmaşıklık Matrisi')
plt.xlabel('Tahmin Edilen Etiketler')
plt.ylabel('Gerçek Etiketler')
plt.show()

## Özellik Önem Sırasını Görselleştirme

In [ ]:
# Özellik önem sırasını belirle ve görselleştir
önemler = en_iyi_rf_modeli.feature_importances_
özellik_önemi_df = pd.DataFrame({
    'Özellik': X.columns,
    'Önem': önemler
}).sort_values(by='Önem', ascending=False)

plt.figure(figsize=(10, 6))
sns.barplot(x='Önem', y='Özellik', data=özellik_önemi_df)
plt.title('Özelliklerin Önem Sırası')
plt.show()

özellik_önemi_df

## İşlenmiş Veriyi ve Özellik Önem Sırasını Kaydetme

In [ ]:
# İşlenmiş veriyi ve özellik önem sırasını CSV dosyası olarak kaydet
data.to_csv('processed_data.csv', index=False)
özellik_önemi_df.to_csv('feature_importance.csv', index=False)

print("Dosyalar kaydedildi:")
print("1. İşlenmiş veri: processed_data.csv")
print("2. Özellik önemi: feature_importance.csv")